## Reinforcement Learning for Predictive Mainteance

- Author: Rajesh Siraskar
- Date: 09-Jan-2022

#### Elective Objectives:
1. Basics: RL
2. Python basics: numpy, pandas and matplotlib
3. Advanced: PyTorch, RL programming, TensorBoard
4. Understand OpenAI gym and create **custom env.** 
5. Put together a RL loop - non-functional/dummy
6. Show training curves in TensorBoard
7. **Propose ideas for a functional model**

#### Approach/Ideas:
1. Action: Is one of No-action, Maintenance-Regular, Maintenance-Special, Replace
2. Cost: Associate cost of taking these: Replace = 100, Special=40, Regular=20
3. After each maintenance: Increase RUL: Replace = x10, Special=x5, Regular=x2
4. Doing regular in downward trend of operation -> RUL increase only 10% regular
5. Reward: Most life, least cost so for example: R = RUL/(Cost+1) i.e. as RUL high and cost low -> reward MOST 
6. What one expects: Regular maintenance done in normal cycle
7. Optimal Policy = When to maintain with least cost

- Running TensorBoard: 
    - conda activate RL_OptimalControl
    - tensorboard --logdir E:\Projects\RL_for_Predictive_Maintenance\tensorboard\.

In [13]:
import gym
import json

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO, DQN

#### Constants

In [14]:
EPISODES_MAX = 4000

BEARING_FAILURE_DATA_FILE = 'data\data.csv'
TRAINING_RESULTS_FILE = 'Test_run_A2C_09-Feb.csv'

## Environment: Custom class based on OpenGym abstract class 
### Bearing failure environment

In [15]:
import random
import math
import json
import gym
from gym import spaces
import pandas as pd
import numpy as np

MAX_LIFE = 470

MAX_VIBRATION = 2
MAX_REWARD = 10
MAX_STEPS = 20000
RUL_WINDOW = 10
LAMBDA = 0.01
INITIAL_RUL = MAX_LIFE

NO_ACTION = 0
MAINTENANCE_REGULAR = 1
MAINTENANCE_SPECIAL = 2
MAINTENANCE_REPLACE = 3

a_cost = []
a_action_recommended = []
a_actions = []
a_action_text = []
a_rul = []
a_rewards = []
a_events = []
a_time = []

class BearingEnv(gym.Env):
    """A Bearing RUL estimating environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df):        
        super(BearingEnv, self).__init__()

        self.df = df
        self.maintenance_cost = 0
        self.maintenance_events = 0
        self.rul = 0
        self.reward = 0
        self.current_action = NO_ACTION
        
        self.reward_range = (0, MAX_REWARD)

        high = np.array(
            [
                MAX_LIFE,
                MAX_VIBRATION,
                MAINTENANCE_REPLACE,
            ],
            dtype=np.float32,
        )
        
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)
        
        self.state = None
            
    def _next_observation(self):      
        frame = np.array([
            self.df.loc[self.current_step, 'RUL'] / MAX_LIFE,
            self.df.loc[self.current_step, 'vibration'] / MAX_VIBRATION,
            self.df.loc[self.current_step, 'temperature'],
        ])
        
        self.rul = self.df.loc[self.current_step: self.current_step, 'RUL'].values[0]
        
        # Append additional data and scale each value to between 0-1
        obs = frame.flatten()
        return obs

    def _take_action(self, action):

        if action == NO_ACTION: # Normal state
            # 1% reduction in life
            # self.rul *= 0.99
            # self.maintenance_cost += 0.1
            action_text = 'None'
            
        elif action == MAINTENANCE_REGULAR: 
            # 1% increase in life
            self.rul *= 1.01
            self.maintenance_cost += 1
            self.maintenance_events += 1
            action_text = 'Maintenance-Regular'

        elif action == MAINTENANCE_SPECIAL: 
            # 5% increase in life
            self.rul *= 1.05
            self.maintenance_cost += 4
            self.maintenance_events += 1
            action_text = 'Maintenance-Special' 
            
        elif action == MAINTENANCE_REPLACE: 
            # Normal state
            self.rul = MAX_LIFE
            self.maintenance_cost += 20
            self.maintenance_events += 1
            action_text = '* REPLACE *' 
            
        #a_actions.append(action)
        a_action_text.append(action_text)
        self.reward = self.rul / (self.maintenance_cost+LAMBDA)
        
        print('{0:<20} | RUL: {1:>8.2f} | Cost: {2:>8.2f} | Maintenance events: {3:>3d} | Reward: {4:>12.3f}'.
              format(action_text, self.rul, self.maintenance_cost, self.maintenance_events, self.reward))
                 
        self.state = (self.rul, self.maintenance_cost, action)

  
    def step(self, action):        
        # Execute one time step within the environment
        self._take_action(action)
        reward = self.reward
        
        if self.current_step >= (len(self.df.loc[:, 'RUL'].values)-1):
            done = True
        else:
            self.current_step += 1
            done = False
                
        if self.rul <= 10:
            done = True
        elif self.rul >= MAX_LIFE:
            done = True
        else:
            done = False

        obs = self._next_observation()
        
        a_time.append(self.current_step)
        a_rewards.append(self.reward)
        a_rul.append(self.rul)
        a_cost.append(self.maintenance_cost)
        a_actions.append(action)
        a_events.append(self.maintenance_events)
        
        # From database extract recommended action
        recommended_action = self.df.loc[self.current_step: self.current_step, 'ACTION_CODE'].values[0]
        a_action_recommended.append(recommended_action)
        
        return obs, reward, done, {}

    def reset(self):
        done = False
        self.state = np.random.uniform(low=-0.0, high=1.0, size=(3,))

        # Set the current step to a random point within the data frame
        self.current_step = 0
        # self.current_step = random.randint(0, len(self.df.loc[:, 'RUL'].values))

        self.rul = self.df.loc[self.current_step: self.current_step, 'RUL'].values[0]
        print('\n --- RESET. Starting RUL = ', self.rul)
        # Reset the state of the environment to an initial state
        self.maintenance_cost = 0
        self.maintenance_events = 0
        self.reward = 0
        self.current_action = NO_ACTION
        
        return np.array(self.state, dtype=np.float32)

    def render(self, mode='human', close=False):
        # Render the environment to the screen
        RUL = self.rul
        
        print('>> {0:<20} | RUL: {1:>8.2f} | Cost: {2:>8.2f} | Reward: {3:>12.3f}'.
              format(self.current_action, self.rul, self.maintenance_cost, self.reward))

### Check environment definition stability

In [16]:
# from stable_baselines3.common.env_checker import check_env
# env = BearingEnv(df)
# check_env(env)

### Read data

In [17]:
df = pd.read_csv(BEARING_FAILURE_DATA_FILE)

In [18]:
EPISODES = EPISODES_MAX
RUN_TEST = 20

### Train the agent

In [19]:
# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: BearingEnv(df)])

model = A2C('MlpPolicy', env, verbose=1, tensorboard_log="./tensorboard/")
#model = DQN('MlpPolicy', env, verbose=1, tensorboard_log="./tensorboard/")
#model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="./tensorboard/")

model.learn(total_timesteps=EPISODES)

Using cpu device

 --- RESET. Starting RUL =  461.331
Logging to ./tensorboard/A2C_6
Maintenance-Special  | RUL:   484.40 | Cost:     4.00 | Maintenance events:   1 | Reward:      120.797

 --- RESET. Starting RUL =  461.331
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
* REPLACE *          | RUL:   470.00 | Cost:    20.00 | Maintenance events:   1 | Reward:       23.488

 --- RESET. Starting RUL =  461.331
* REPLACE *          | RUL:   470.00 | Cost:    20.00 | Maintenance events:   1 | Reward:       23.488

 --- RESET. Starting RUL =  461.331
Maintenance-Special  | RUL:   484.40 | Cost:     4.00 | Maintenance events:   1 | Reward:      120.797

 --- RESET. Starting RUL =  461.331
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
Maintenance-Regular  | RUL:   460.89 | Cost:     1.00 | Maintenance events:   1 | Reward:      456.331
None                 | RUL:   451.33 | Cost:  

None                 | RUL:   251.33 | Cost:    10.00 | Maintenance events:   4 | Reward:       25.108
None                 | RUL:   246.33 | Cost:    10.00 | Maintenance events:   4 | Reward:       24.608
None                 | RUL:   241.33 | Cost:    10.00 | Maintenance events:   4 | Reward:       24.109
* REPLACE *          | RUL:   470.00 | Cost:    30.00 | Maintenance events:   5 | Reward:       15.661

 --- RESET. Starting RUL =  461.331
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
None                 | RUL:   456.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45633.100
None                 | RUL:   451.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45133.100
* REPLACE *          | RUL:   470.00 | Cost:    20.00 | Maintenance events:   1 | Reward:       23.488

 --- RESET. Starting RUL =  461.331
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100

None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
None                 | RUL:   456.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45633.100
None                 | RUL:   451.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45133.100
None                 | RUL:   446.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44633.100
None                 | RUL:   441.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44133.100
None                 | RUL:   436.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    43633.100
None                 | RUL:   431.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    43133.100
None                 | RUL:   426.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    42633.100
None                 | RUL:   421.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    42133.100
None                 | RUL:   416.33 | Cost:     0.00 | Maintenance event

None                 | RUL:   426.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      422.110
None                 | RUL:   421.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      417.159
None                 | RUL:   416.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      412.209
None                 | RUL:   411.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      407.258
None                 | RUL:   406.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      402.308
None                 | RUL:   401.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      397.357
None                 | RUL:   396.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      392.407
None                 | RUL:   391.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      387.456
Maintenance-Regular  | RUL:   390.19 | Cost:     2.00 | Maintenance events:   2 | Reward:      194.127
None                 | RUL:   381.33 | Cost:     2.00 | Maintenance event

None                 | RUL:   396.33 | Cost:     4.00 | Maintenance events:   1 | Reward:       98.836
None                 | RUL:   391.33 | Cost:     4.00 | Maintenance events:   1 | Reward:       97.589
None                 | RUL:   386.33 | Cost:     4.00 | Maintenance events:   1 | Reward:       96.342
None                 | RUL:   381.33 | Cost:     4.00 | Maintenance events:   1 | Reward:       95.095
None                 | RUL:   376.33 | Cost:     4.00 | Maintenance events:   1 | Reward:       93.848
Maintenance-Regular  | RUL:   375.04 | Cost:     5.00 | Maintenance events:   2 | Reward:       74.859
None                 | RUL:   366.33 | Cost:     5.00 | Maintenance events:   2 | Reward:       73.120
None                 | RUL:   361.33 | Cost:     5.00 | Maintenance events:   2 | Reward:       72.122
None                 | RUL:   356.33 | Cost:     5.00 | Maintenance events:   2 | Reward:       71.124
None                 | RUL:   351.33 | Cost:     5.00 | Maintenance event

...

None                 | RUL:   190.91 | Cost:     0.00 | Maintenance events:   0 | Reward:    19091.100
None                 | RUL:   187.38 | Cost:     0.00 | Maintenance events:   0 | Reward:    18738.100
None                 | RUL:   183.94 | Cost:     0.00 | Maintenance events:   0 | Reward:    18394.100
None                 | RUL:   180.53 | Cost:     0.00 | Maintenance events:   0 | Reward:    18053.100
None                 | RUL:   177.11 | Cost:     0.00 | Maintenance events:   0 | Reward:    17711.100
None                 | RUL:   173.63 | Cost:     0.00 | Maintenance events:   0 | Reward:    17363.100
None                 | RUL:   170.02 | Cost:     0.00 | Maintenance events:   0 | Reward:    17002.100
None                 | RUL:   166.22 | Cost:     0.00 | Maintenance events:   0 | Reward:    16622.100
None                 | RUL:   162.13 | Cost:     0.00 | Maintenance events:   0 | Reward:    16213.100
None                 | RUL:   157.58 | Cost:     0.00 | Maintenance event

None                 | RUL:   170.02 | Cost:     0.00 | Maintenance events:   0 | Reward:    17002.100
None                 | RUL:   166.22 | Cost:     0.00 | Maintenance events:   0 | Reward:    16622.100
None                 | RUL:   162.13 | Cost:     0.00 | Maintenance events:   0 | Reward:    16213.100
None                 | RUL:   157.58 | Cost:     0.00 | Maintenance events:   0 | Reward:    15758.100
None                 | RUL:   152.30 | Cost:     0.00 | Maintenance events:   0 | Reward:    15230.100
None                 | RUL:   145.69 | Cost:     0.00 | Maintenance events:   0 | Reward:    14569.100
None                 | RUL:   136.00 | Cost:     0.00 | Maintenance events:   0 | Reward:    13600.000
None                 | RUL:   136.00 | Cost:     0.00 | Maintenance events:   0 | Reward:    13600.000
None                 | RUL:   135.00 | Cost:     0.00 | Maintenance events:   0 | Reward:    13500.000
None                 | RUL:   133.00 | Cost:     0.00 | Maintenance event

None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
None                 | RUL:   456.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45633.100
None                 | RUL:   451.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45133.100
None                 | RUL:   446.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44633.100
None                 | RUL:   441.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44133.100
None                 | RUL:   436.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    43633.100
None                 | RUL:   431.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    43133.100
None                 | RUL:   426.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    42633.100
None                 | RUL:   421.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    42133.100
None                 | RUL:   416.33 | Cost:     0.00 | Maintenance event

None                 | RUL:   446.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      441.912
None                 | RUL:   441.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      436.961
None                 | RUL:   436.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      432.011
None                 | RUL:   431.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      427.060
None                 | RUL:   426.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      422.110
None                 | RUL:   421.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      417.159
None                 | RUL:   416.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      412.209
None                 | RUL:   411.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      407.258
None                 | RUL:   406.33 | Cost:     1.00 | Maintenance events:   1 | Reward:      402.308
None                 | RUL:   401.33 | Cost:     1.00 | Maintenance event

None                 | RUL:    31.00 | Cost:     1.00 | Maintenance events:   1 | Reward:       30.693
None                 | RUL:    16.00 | Cost:     1.00 | Maintenance events:   1 | Reward:       15.842
None                 | RUL:     0.00 | Cost:     1.00 | Maintenance events:   1 | Reward:        0.000

 --- RESET. Starting RUL =  461.331
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
None                 | RUL:   456.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45633.100
None                 | RUL:   451.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45133.100
None                 | RUL:   446.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44633.100
None                 | RUL:   441.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44133.100
None                 | RUL:   436.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    43633.100
None                 | RUL:   431.33

### Store results in a Pandas data-frame

In [20]:
df_results = pd.DataFrame({'Time-step':a_time[:], 'Recommended_Action':a_action_recommended[:], 'Action':a_action_text[:], 
                           'Action_Code':a_actions[:], 'Rewards':a_rewards[:], 'RUL':a_rul[:], 
                           'MaintenanceEvents':a_events[:], 'Cost':a_cost[:]})
df_results.head()

,Time-step,Recommended_Action,Action,Action_Code,Rewards,RUL,MaintenanceEvents,Cost
0,1,0,Maintenance-Special,2,120.797394,456.331,1,4
1,1,0,None,0,46133.100000,456.331,0,0
2,2,0,* REPLACE *,3,23.488256,451.331,1,20
3,1,0,* REPLACE *,3,23.488256,456.331,1,20
4,1,0,Maintenance-Special,2,120.797394,456.331,1,4


In [21]:
df_results.Rewards = df_results.Rewards/df_results.Rewards.max()
df_results.RUL = df_results.RUL/df_results.RUL.max()
df_results.Cost = df_results.Cost/df_results.Cost.max()

### Convert to CSV for Analysis notebook

In [22]:
df_results.to_csv(TRAINING_RESULTS_FILE)

### Test agent

In [23]:
obs = env.reset()
for i in range(20):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()


 --- RESET. Starting RUL =  461.331
None                 | RUL:   461.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    46133.100
>> 0                    | RUL:   456.33 | Cost:     0.00 | Reward:    46133.100
None                 | RUL:   456.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45633.100
>> 0                    | RUL:   451.33 | Cost:     0.00 | Reward:    45633.100
None                 | RUL:   451.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    45133.100
>> 0                    | RUL:   446.33 | Cost:     0.00 | Reward:    45133.100
None                 | RUL:   446.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44633.100
>> 0                    | RUL:   441.33 | Cost:     0.00 | Reward:    44633.100
None                 | RUL:   441.33 | Cost:     0.00 | Maintenance events:   0 | Reward:    44133.100
>> 0                    | RUL:   436.33 | Cost:     0.00 | Reward:    44133.100
None                 | RUL:   436.33 | Cost:    